In [1]:
from pyiron_atomistics import Project
from pyscal_rdf import StructureGraph
from rdfjobs.lammps.lammps import RDFLammps
from tqdm.notebook import trange

In [2]:
import numpy as np

In [3]:
pr = Project("test_100")

In [4]:
s = StructureGraph(store=pr)

In [5]:
elements = ["Fe"]
#elements = ["Fe"]
reps = 2

In [6]:
for x in trange(len(elements)):
    element = elements[x]
    #create structure
    struct = s.create_element(element, repetitions=(reps, reps, reps))
    #minimise this
    job = pr.create_job(job_type=RDFLammps, 
                        job_name=f'{element}_j_1', 
                        delete_existing_job=True)
    job.structure = struct
    job.server.cores = 2
    if element in ["Mg", "Zn"]:
        job.potential = job.list_potentials()[1]
    else:
        job.potential = job.list_potentials()[0]
    job.calc_minimize()
    job.run()
    energy_bulk = job.output.energy_tot[-1]
    number_atoms = struct.natoms

    #collect the minimised structure and add it to graph
    min_struct = job.get_structure_as_system()
    s.add_structure_to_graph(min_struct)
    #delete one atom
    del min_struct[np.random.randint(0, min_struct.natoms)]

    #now minimise this structure
    job = pr.create_job(job_type=RDFLammps, 
                        job_name=f'{element}_j_2', 
                        delete_existing_job=True)
    job.structure = min_struct
    if element in ["Mg", "Zn"]:
        job.potential = job.list_potentials()[1]
    else:
        job.potential = job.list_potentials()[0]
    job.calc_minimize()
    job.run()
    energy_vac = job.output.energy_tot[-1]

    #now calculate vacancy formation energy
    form_energy = energy_vac - ((number_atoms-1)/number_atoms)*energy_bulk
    #add it to the graph
    s.add_calculated_quantity("VacancyFormationEnergy", 
                              form_energy, 
                              unit="EV",
                              sample=job._final_sample)

  0%|          | 0/1 [00:00<?, ?it/s]

Fe


2023-07-04 11:38:42,454 - pyiron_log - WARNING - Job aborted
2023-07-04 11:38:42,455 - pyiron_log - WARNING - [mpiexec@cmleo26] match_arg (lib/utils/args.c:166): unrecognized argument oversubscribe
[mpiexec@cmleo26] HYDU_parse_array (lib/utils/args.c:181): argument matching returned error
[mpiexec@cmleo26] parse_args (mpiexec/get_parameters.c:315): error parsing input array
[mpiexec@cmleo26] HYD_uii_mpx_get_parameters (mpiexec/get_parameters.c:47): unable to parse user arguments
[mpiexec@cmleo26] main (mpiexec/mpiexec.c:49): error parsing parameters



The job Fe_j_1 was saved and received the ID: 718


RuntimeError: Job aborted

In [7]:
form_energy, s.n_samples

(0.15976862346362708, 28)